In [175]:
### This is ML model to predict film ratings!
import pandas as pd

In [208]:
df = pd.read_csv('../CrawlData/all_data/final_data_movie.csv')

df = df.drop(columns=['Unnamed: 0.1','Unnamed: 0']).reset_index(drop=True)

In [209]:
df['Genre'].fillna('DramaDrama')
df = df.drop(columns='Licensors')

In [210]:
df['Rating'] = df['Rating'].fillna('G - All Ages')

In [211]:
df.head(3)

,Title,Genre,Aired,Studios,Theme,Ranked,Demographic,Source,Broadcast,Members,...,Episodes,Type,Producers,Popularity,Duration,Rating,Premiered,Status,score,review_film
0,Frieren: Beyond Journey's End,"Adventure, Drama, Fantasy","Sep 29, 2023 to Mar 22, 2024",Madhouse,all,1,Shounen,Manga,Fridays at 23:00 (JST),725256,...,28,TV,"Aniplex, Dentsu, Shogakukan-Shueis...",277,24 min. per ep.,PG-13 - Teens 13 or older,Fall 2023,Finished Airing,9.37,During their decade-long quest to defeat the D...
1,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy","Apr 5, 2009 to Jul 4, 2010",Bones,Military,2,Shounen,Manga,"Sundays at 17:00 (JST) ', 'Aniplex, ...",3346066,...,64,TV,"Aniplex, Square Enix, Mainichi Bro...",3,24 min. per ep.,R - 17+ (violence & profanity),Spring 2009,Finished Airing,9.09,After a horrific alchemy experiment goes wrong...
2,Steins;Gate,"Drama, Sci-Fi, Suspense","Apr 6, 2011 to Sep 14, 2011",White Fox,"Psychological, Time , Time Travel, Time Travel...",3,All,Visual novel,Wednesdays at 02:05 (JST),2564359,...,24,TV,"Frontier Works, Media Factory, Kad...",13,24 min. per ep.,PG-13 - Teens 13 or older,Spring 2011,Finished Airing,9.07,Eccentric scientist Rintarou Okabe has a never...


In [212]:
from collections import defaultdict
my_d = defaultdict(int)
for i in df['Rating']:
    if i in my_d:
        my_d[i] += 1
    else:my_d[i] = 1

In [213]:
#! Nếu như dữ liệu predict ra là: teens 13, all age -> phim hay!
# còn lại thì là dở

In [214]:
# pd.api.types.is_float_dtype(df['Genre'])

In [215]:
# text preprocessing
from nltk import word_tokenize
text = list(df['Genre'])
# preprocessing loop
#print(text)
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

corpus = []
#! text is a paragraph!
for i in range(len(text)):
    
    r = r.lower()
    
    #! stop word
    
    # r = [word for word in r if word not in stopwords.words('english')]
    # r = ''.join(r)
    # word_token = word_tokenize(r)
    # #print(word_token)
    # r = [lemmatizer.lemmatize(word) for word in word_token]
    # r = ' '.join(r)
    #r = r.split()
    corpus.append(r)

#assign corpus to data['text']
df['Genre'] = corpus


In [216]:
df['Genre'][0]

'wr wnnng , wr wnnngwr , r , fn , upene'

In [217]:
#! Để có thể classify được text thì bằng cách nào đó phải encode ra vector? 
#! text processing time!
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

In [218]:
from sklearn.model_selection import train_test_split
X_train, X_test,Y_train,Y_test = train_test_split(df['Demographic'],df['Rating'],random_state=42,test_size=0.4)
cv = CountVectorizer()
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)
X_train_cv.shape

(2319, 6)

In [189]:
X_test

2323           All
1419       Shounen
2477    JoseiJosei
1583         Shojo
952          Shojo
           ...    
2464           All
998          Shojo
2022        Seinen
3021           All
1991           All
Name: Demographic, Length: 1546, dtype: object

In [190]:
cv.get_feature_names_out()

array(['all', 'joseijosei', 'kids', 'seinen', 'shojo', 'shounen'],
      dtype=object)

In [191]:
log_model = LogisticRegression()

In [192]:
X_train_cv.shape

(2319, 6)

In [193]:
# Y_train.shape

In [194]:
log_model.fit(X_train_cv,Y_train)

LogisticRegression()

In [195]:
X_test_cv.shape

(1546, 6)

In [196]:
pred = log_model.predict(X_test_cv)

In [197]:
from sklearn import metrics

metrics.confusion_matrix(y_pred=pred,y_true=Y_test)

array([[   0,    7,   97,    0,    0],
       [   0,   30,   41,    0,    0],
       [   0,    1, 1034,    0,    0],
       [   0,    0,  274,    0,    0],
       [   0,    0,   62,    0,    0]], dtype=int64)

In [198]:
metrics.accuracy_score(y_true=Y_test,y_pred=pred)
#metrics.f1_score(y_true=Y_test,y_pred=pred)

0.6882276843467011

In [199]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=150, random_state=0)

clf.fit(X_train_cv,Y_train)

RandomForestClassifier(max_depth=150, random_state=0)

In [200]:
pred = clf.predict(X_test_cv)

In [201]:
metrics.accuracy_score(y_true=Y_test,y_pred=pred)

0.6882276843467011

In [202]:
print(X_test_cv[2])
print(log_model.predict(X_test_cv[300]))

  (0, 1)	1
['PG-13 - Teens 13 or older']


In [203]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(X_train_cv,Y_train)
pred = DT.predict(X_test_cv)
metrics.accuracy_score(Y_test,pred)



0.6882276843467011

In [204]:
import pickle

pickle.dump(DT,open('./test2.pkl','wb'))

In [205]:
#! Để update SVM thì mình có thể dùng trick chọn kernel function cho hiệu quả
#! https://techntales.medium.com/what-is-svm-and-how-does-it-work-b20d612af9e2#:~:text=SVM%20works%20by%20finding%20a,three%20features%2C%20and%20so%20on.
#! maximum margin seperating


from sklearn import svm



h = 0.02  # step size in the mesh
SVM = svm.SVC(kernel='poly',gamma=1.00)
SVM.fit(X_train_cv,Y_train)
pred = SVM.predict(X_test_cv)
metrics.accuracy_score(Y_test,pred)

0.6882276843467011

In [206]:
#! Làm sao mà mình biết được dữ liệu của mình hình dáng thế nào để vẽ được đường phù hợp? 


In [207]:
from sklearn.linear_model import SGDClassifier
sgd = SGDClassifier()
sgd.fit(X_train_cv,Y_train)
pred = sgd.predict(X_test_cv)
metrics.accuracy_score(Y_test,pred)

0.6882276843467011